# Loop 4 Analysis: Finding a Path Forward

We've been stuck at 70.734327 for 4 experiments. The evaluator identified that:
1. The cpp_parallel_sa configs score 67.77 but have massive overlaps (183-198 N values)
2. Simple repair doesn't work - configs are too densely packed
3. We need to either re-optimize with collision constraints OR find a different approach

Let's analyze:
1. What makes the 67.77 configs better (even with overlaps)?
2. Can we use the jonathanchan C++ optimizer with proper collision constraints?
3. What's the theoretical lower bound for this problem?

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from numba import njit
import math
import matplotlib.pyplot as plt

os.chdir('/home/code')

@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def get_bbox(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    side = get_bbox(xs, ys, degs, tx, ty)
    return side * side / n

tx, ty = make_polygon_template()

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        configs[n] = {'x': xs, 'y': ys, 'deg': ds}
    return configs

print('Functions defined')

In [ ]:
# Load baseline
baseline = load_submission('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv')

# Calculate baseline scores per N
baseline_scores = {}
for n in range(1, 201):
    if n in baseline:
        c = baseline[n]
        baseline_scores[n] = score_group(c['x'], c['y'], c['deg'], tx, ty)

total_baseline = sum(baseline_scores.values())
print(f'Baseline total score: {total_baseline:.6f}')
print(f'Target: 68.931058')
print(f'Gap: {total_baseline - 68.931058:.6f}')

In [ ]:
# Analyze score distribution by N
n_values = list(range(1, 201))
scores = [baseline_scores[n] for n in n_values]

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.bar(n_values, scores)
plt.xlabel('N')
plt.ylabel('Score (s²/n)')
plt.title('Score contribution by N')

plt.subplot(1, 2, 2)
cumulative = np.cumsum(scores)
plt.plot(n_values, cumulative)
plt.xlabel('N')
plt.ylabel('Cumulative Score')
plt.title('Cumulative score')
plt.tight_layout()
plt.savefig('exploration/score_distribution.png', dpi=100)
plt.show()

# Top 10 N values by score contribution
sorted_scores = sorted(baseline_scores.items(), key=lambda x: x[1], reverse=True)
print('\nTop 10 N values by score contribution:')
for n, s in sorted_scores[:10]:
    print(f'  N={n}: {s:.6f}')

In [ ]:
# Calculate theoretical lower bound
# For a single tree at 45 degrees, the bounding box is minimized
# Tree dimensions: width=0.7, height=1.0 (from -0.2 to 0.8)
# At 45 degrees, the diagonal is sqrt(0.7^2 + 1.0^2) = 1.22
# But the actual bounding box depends on the polygon shape

# For N=1, the optimal is a single tree at 45 degrees
# Let's calculate what that score would be
test_x = np.array([0.0])
test_y = np.array([0.0])
test_deg = np.array([45.0])
n1_score = score_group(test_x, test_y, test_deg, tx, ty)
print(f'N=1 at 45 degrees: {n1_score:.6f}')
print(f'N=1 baseline: {baseline_scores[1]:.6f}')

# The baseline N=1 is already optimal
print(f'\nN=1 is optimal: {abs(n1_score - baseline_scores[1]) < 1e-6}')

In [ ]:
# Check if the jonathanchan C++ optimizer source is available
import subprocess

# Check if we can compile the C++ optimizer
result = subprocess.run(['which', 'g++'], capture_output=True, text=True)
print(f'g++ available: {result.returncode == 0}')

# Check OpenMP support
result = subprocess.run(['g++', '--version'], capture_output=True, text=True)
print(f'g++ version: {result.stdout.split(chr(10))[0]}')

# Check if we have the jonathanchan kernel source
kernel_path = '/home/code/research/kernels/jonathanchan_santa25-ensemble-sa-fractional-translation/'
print(f'\nJonathanchan kernel available: {os.path.exists(kernel_path)}')

In [ ]:
# Let's try to compile and run the jonathanchan C++ optimizer
# First, extract the C++ code from the notebook

import json

with open('/home/code/research/kernels/jonathanchan_santa25-ensemble-sa-fractional-translation/santa25-ensemble-sa-fractional-translation.ipynb', 'r') as f:
    nb = json.load(f)

# Find the C++ code cell
cpp_code = None
for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if 'sa_v3' in source and '#include' in source:
            cpp_code = source
            break

if cpp_code:
    print('Found C++ code in jonathanchan kernel')
    print(f'Code length: {len(cpp_code)} characters')
    # Save to file
    with open('/home/code/experiments/sa_v3_parallel.cpp', 'w') as f:
        f.write(cpp_code)
    print('Saved to /home/code/experiments/sa_v3_parallel.cpp')
else:
    print('C++ code not found')

In [ ]:
# Compile the C++ optimizer
import subprocess

result = subprocess.run(
    ['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', 
     '-o', '/home/code/experiments/sa_v3_parallel', 
     '/home/code/experiments/sa_v3_parallel.cpp'],
    capture_output=True, text=True
)

if result.returncode == 0:
    print('Compilation successful!')
else:
    print(f'Compilation failed:')
    print(result.stderr[:2000])

In [ ]:
# If compilation succeeded, let's run the optimizer on our baseline
import shutil

if os.path.exists('/home/code/experiments/sa_v3_parallel'):
    # Copy baseline to working directory
    shutil.copy('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv',
                '/home/code/experiments/submission_best.csv')
    
    # Run optimizer with moderate parameters
    os.chdir('/home/code/experiments')
    result = subprocess.run(
        ['./sa_v3_parallel', '-i', 'submission_best.csv', '-n', '5000', '-r', '3'],
        capture_output=True, text=True, timeout=600
    )
    print('Output:')
    print(result.stdout[:3000])
    if result.stderr:
        print('Errors:')
        print(result.stderr[:1000])
else:
    print('Optimizer not compiled')